<a href="https://colab.research.google.com/github/saishshinde15/NLP/blob/main/Sentiment_analysis_HuggingFaceipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
dataset=pd.read_csv('/content/file.csv')

In [3]:
dataset

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad
...,...,...,...
219289,219289,Other Software Projects Are Now Trying to Repl...,bad
219290,219290,I asked #ChatGPT to write a #NYE Joke for SEOs...,good
219291,219291,chatgpt is being disassembled until it can onl...,bad
219292,219292,2023 predictions by #chatGPT. Nothing really s...,bad


In [5]:
dataset=dataset.drop(['Unnamed: 0'],axis=1)

In [6]:
dataset

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad
...,...,...
219289,Other Software Projects Are Now Trying to Repl...,bad
219290,I asked #ChatGPT to write a #NYE Joke for SEOs...,good
219291,chatgpt is being disassembled until it can onl...,bad
219292,2023 predictions by #chatGPT. Nothing really s...,bad


In [7]:
dataset.isnull().sum()

tweets    0
labels    0
dtype: int64

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219294 entries, 0 to 219293
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweets  219294 non-null  object
 1   labels  219294 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [9]:
dataset['labels'].unique()

array(['neutral', 'good', 'bad'], dtype=object)

In [10]:
X=dataset['tweets']

In [11]:
X

0         ChatGPT: Optimizing Language Models for Dialog...
1         Try talking with ChatGPT, our new AI system wh...
2         ChatGPT: Optimizing Language Models for Dialog...
3         THRILLED to share that ChatGPT, our new model ...
4         As of 2 minutes ago, @OpenAI released their ne...
                                ...                        
219289    Other Software Projects Are Now Trying to Repl...
219290    I asked #ChatGPT to write a #NYE Joke for SEOs...
219291    chatgpt is being disassembled until it can onl...
219292    2023 predictions by #chatGPT. Nothing really s...
219293     From ChatGPT, neat stuff https://t.co/qjjUF2Z2m0
Name: tweets, Length: 219294, dtype: object

In [14]:
y=dataset['labels'].values

In [15]:
y

array(['neutral', 'good', 'neutral', ..., 'bad', 'bad', 'neutral'],
      dtype=object)

In [16]:


from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [17]:
y

array([2, 1, 2, ..., 0, 0, 2])

In [18]:


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [19]:
import tensorflow as tf
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer

In [20]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = TFRobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: bad, neutral, good


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
X_train_tokens = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=512, return_tensors="tf")
X_test_tokens = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=512, return_tensors="tf")

In [22]:
rnn = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer), output_dim=50, input_length=X_train_tokens.input_ids.shape[1]),
    tf.keras.layers.LSTM(units=32, activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.LSTM(units=64, activation='tanh'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=3, activation='softmax')  # 3 output units for bad, neutral, good
])

In [23]:
rnn.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [24]:

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=2, monitor='val_loss', restore_best_weights=True)


In [ ]:
history = rnn.fit(X_train_tokens.input_ids, y_train, epochs=1, batch_size=32,
                   validation_data=(X_test_tokens.input_ids, y_test), callbacks=[early_stopping])

2381/5483 [============>.................] - ETA: 31:53 - loss: 1.0482 - accuracy: 0.4909